<a href="https://colab.research.google.com/github/robert-sturrock/finance-analysis/blob/master/financial_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U seaborn

    100% |████████████████████████████████| 215kB 26.6MB/s 
  Found existing installation: seaborn 0.7.1
    Uninstalling seaborn-0.7.1:
      Successfully uninstalled seaborn-0.7.1


In [0]:
!pip install dash==0.35.1  # The core dash backend
!pip install dash-html-components==0.13.4  # HTML components
!pip install dash-core-components==0.42.1  # Supercharged components
!pip install dash-table==3.1.11  # Interactive DataTable component (new!)

    100% |████████████████████████████████| 942kB 1.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/5a/c8/61/4daf9b3c3b33e8037294f3e694b84b2a6e04cf1d7ab6724fa5
  Stored in directory: /root/.cache/pip/wheels/96/32/88/a1f6d9dd3c29570ab3a8acc0d556b3b20abcf3c623c868ce0a
  Stored in directory: /root/.cache/pip/wheels/09/d2/49/0d140766d5ee1ef4ce271d302a5d30cf3d18c5b507e48c77f3
Successfully built dash flask-compress dash-renderer
    100% |████████████████████████████████| 532kB 23.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/23/32/a4/73402d8718ad5e92e9d47dcfa088be5481961e2b562ea09803
Successfully built dash-html-components
    100% |████████████████████████████████| 4.1MB 1.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/4d/0c/74/570bc5933e80be75300dfe1004b6541e0b5c8ef2b955dc7904
Successfully built dash-core-components
    100% |████████████████████████████████| 307kB 26.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/5e/aa/b2/f297562c61542573048d6a9ec49c57c37c2a37

In [0]:
import numpy as np
import matplotlib as m
import matplotlib.pyplot as plt
import nltk as n
import re as re
import plotly
import pandas as pd
import unicodedata 
import requests
import seaborn as sns
from pandas.compat import StringIO
from pandas.compat import BytesIO
from datetime import datetime
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='robertsturrock', api_key='cmvuePKlxyspZ3sMg1tt')

%matplotlib inline

from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import Contours, Histogram2dContour, Marker, Scatter



In [0]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
            },
          });
        </script>
        '''))

## Load financial data
Take in 2018 Monzo csv


In [0]:
url = "https://raw.githubusercontent.com/robert-sturrock/finance-analysis/master/Monzo%202018%20-%20Data.csv"
r = requests.get(url).content

In [0]:
df = pd.read_csv(url)

In [0]:
df.head()

,id,date,created,amount,currency,local_amount,local_currency,category,emoji,description,address,notes,alt_category,category_comb,exclude
0,tx_00009bnW4wU2keVCipi6PB,2018-10,2018-10-17T01:00:32Z,-18.00,GBP,-18.00,GBP,bills,NaN,623394420110,NaN,623394420110,NaN,bills,0
1,tx_00009cu5jwKKwmhut5S1nF,2018-11,2018-11-19T03:00:30Z,-18.00,GBP,-18.00,GBP,bills,NaN,623394420110,NaN,623394420110,NaN,bills,0
2,tx_00009dq8DIcib5K4iB4Fub,2018-12,2018-12-17T03:01:27Z,-18.00,GBP,-18.00,GBP,bills,NaN,623394420110,NaN,623394420110,NaN,bills,0
3,tx_00009bGNoeADsvHw3ct2e1,2018-10,2018-10-01T01:22:29Z,-22.95,GBP,-22.95,GBP,bills,NaN,623394420110,NaN,623394420110,NaN,bills,0
4,tx_00009dCARIgpZjtrMIBNir,2018-11,2018-11-27T20:17:57Z,-37.88,GBP,-37.88,GBP,bills,💳,AQUA MASTERCARD ECOM MORLEY GBR,NaN,NaN,NaN,credit_card,0


In [0]:
# Create date variables

# Create a column with the timestamp 
df['ymd'] = df['created'].str.extract('(\d\d\d\d\-\d\d-\d\d)T.*Z', expand=True)

# Make date 
df['ymd'] =  pd.to_datetime(df['ymd'])


## Basic plots

Looking at my financial data


In [0]:
# Create density plot function
def create_multi_distribution(df, group, value, title):
  
  # Iterate through the senders
  for item in df[group].unique():

      # Subset to the person
      subset = df[df[group] == item]

      # Draw the density plot
      sns.distplot(subset[value], hist = False, kde = True,
                   kde_kws = {'linewidth': 2},
                   label = group)

  # Plot formatting
  plt.legend(prop={'size': 10}, title = 'Airline')
  plt.title(title)
  plt.show()
  

In [0]:
configure_plotly_browser_state()
init_notebook_mode(connected=False)

# Create a timeseries plot of spending 
dff = df.groupby('ymd').agg({'amount':'sum'})

data = [go.Scatter(x=dff.index, y=dff['amount'])]

py.iplot(data, filename = 'time-series-simple')

In [0]:
configure_plotly_browser_state()
init_notebook_mode(connected=False)

# Create a timeseries plot of spending by category 
dff = df[df['category']=='eating_out'].groupby(['ymd','category']).agg({'amount':'sum'}).reset_index()

data = [go.Bar(x=dff.ymd, y=-dff['amount'])] 

py.iplot(data, filename = 'time-series-simple')

In [0]:
# Multiple entry chart

#data = [go.Bar(x=dff.ymd, y=-dff['amount'])]
dff = df[df['category'].isin(['eating_out','bills'])].groupby(['ymd','category']).agg({'amount':'sum'}).reset_index()
for category in dff['category'].unique(): 
  print(category)


data = []
for category in dff['category'].unique():
  dff_tmp = dff[dff['category']==category]
  data.append(
      go.Bar(x=dff_tmp.ymd, y=-dff_tmp['amount'], name = category)
  )


layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar')



In [0]:
def make_list(item):
  return(item if isinstance(item, list) else [item])

In [0]:
my_text=['(sepal length: '+'{:.2f}'.format(sl)+', sepal width:'+'{:.2f}'.format(sw)+')'+
  '<br>(petal length: '+'{:.2f}'.format(pl)+', petal width:'+'{:.2f}'.format(pw)+')'
  for sl, sw, pl, pw in zip(list(df['sepal length (cm)']), list(df['sepal width (cm)']),
                           list(df['petal length (cm)']), list(df['petal width (cm)'])) ] 

In [0]:
def create_multi_bar(df, categories):
  dff = df[df['category'].isin(make_list(categories))].groupby(['ymd','category','description']).agg({'amount':'sum'}).reset_index()
 

  data = []
  for category in dff['category'].unique():
    dff_tmp = dff[dff['category']==category]
    my_text = ['(Amount: '+'{:.2f}'.format(am)+'<br> Description: '+desc+')' for am, desc in zip(list(dff_tmp['amount']),list(dff_tmp['description']))]
    data.append(
        go.Bar(x=dff_tmp.ymd, y=-dff_tmp['amount'], text = my_text, hoverinfo = 'text', name = category)
    )


  layout = go.Layout(
      barmode='group'
  )

  fig = go.Figure(data=data, layout=layout)
  return(py.iplot(fig, filename='grouped-bar'))

In [0]:
create_multi_bar(df, ['eating_out','groceries'])



In [0]:
#### Create average per month for categories that are in play 

# Create a date range 
all_dates = pd.DataFrame({'ymd': pd.date_range(start='1/1/2018', end='31/12/2018')})
all_dates['dummy'] = 1

# Create all categories 
categories = df['category'].unique()
categories = categories[categories != 'NaN']
all_categories = pd.DataFrame({'category':categories})
all_categories['dummy'] = 1

# Create all dates and categories  
df_dates = pd.merge(all_dates, all_categories, how='outer', on = 'dummy')

# Join data to actuals 
df_dates = pd.merge(df_dates, df, how='left', on = ['ymd','category'])

# Change all NaN in amount to 0s, note the use of "inplace=True"
df_dates['amount'].fillna(0, inplace=True)
df_dates.loc[df_dates['category'] == 'eating_out']

# Create data
categories = ['eating_out', 'transport','groceries','shopping']
dff = df_dates[df_dates['category'].isin(make_list(categories))].groupby(['ymd','category']).agg({'amount':'sum'}).reset_index()
dff.loc[dff['category']=='eating_out']

# Create average 
test = dff.groupby(['category'])['amount'].rolling(window=30).sum().rolling(window=5).mean().reset_index(0, drop=True)

# Join back to dataset 
dff['rolling_avg'] = test

# Plot
data = []
for category in np.sort(dff['category'].unique()):
  dff_tmp = dff[dff['category']==category]
  data.append(
      go.Scatter(x=dff_tmp.ymd, y=-dff_tmp['rolling_avg'], name = category)
  )



layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar')

In [0]:
dff.query("category == 'eating_out' & ymd > '2018-05-20'")

In [0]:
df.to_dict('records')

In [0]:
test.head()

In [0]:
# 

# Create data
categories = ['eating_out', 'transport']
dff = df[df['category'].isin(make_list(categories))].groupby(['ymd','category','description']).agg({'amount':'sum'}).reset_index()

# Create average 
test = dff.groupby(['category'])['amount'].rolling(window=15, win_type).mean().reset_index(0, drop=True)
#test = dff.rolling(window=5).mean()
test

# Join back to dataset 
dff['rolling_avg'] = test



# # Plot
# data = []
# for category in dff['category'].unique():
#   dff_tmp = dff[dff['category']==category]
#   data.append(
#       go.Scatter(x=dff_tmp.ymd, y=-dff_tmp['rolling_avg'], name = category)
#   )


# layout = go.Layout(
#     barmode='group'
# )

# fig = go.Figure(data=data, layout=layout)
# py.iplot(fig, filename='grouped-bar')



In [0]:
def create_multi_bar_data(df, categories):
  dff = df[df['category'].isin(make_list(categories))].groupby(['ymd','category']).agg({'amount':'sum'}).reset_index()

  data = []
  for category in dff['category'].unique():
    dff_tmp = dff[dff['category']==category]
    data.append(
        go.Bar(x=dff_tmp.ymd, y=-dff_tmp['amount'], name = category)
    )
    return(data)

In [0]:
check = create_multi_bar_data(df, 'eating_out')
check

In [0]:
df_test = df[df['category']=='eating_out'].loc[:,["ymd",'amount','category','description']]
df_test.head()

In [0]:
dff = df[df.category=='eating_out']
dff.head()

In [0]:
dff = df[df['category']=='eating_out'].groupby(['ymd','category']).agg({'amount':'sum'})
type(dff)

In [0]:
df[df['category'].isin(['bills'])]

In [0]:
df = 'list'
if isinstance(df, list): 
  df = df
else: 
  df = [df]

print(df)

In [0]:
dff.index

In [0]:
dff = dff.reset_index()

In [0]:
dff.columns

In [0]:
dff = df.groupby('ymd').agg({'amount':'sum'})

dff.columns

In [0]:
# Create a summary table 
agg_dict = {
    "local_amount": ["mean","sum","count"], 
    }


su = df.groupby(['date','category_comb']).agg(agg_dict)
print(su)

In [0]:
# load an example dataset
from vega_datasets import data
cars = data.cars()

# plot the dataset, referencing dataframe column names
import altair as alt
alt.Chart(cars).mark_point().encode(
  x='Horsepower',
  y='Miles_per_Gallon',
  color='Origin'
).interactive()

In [0]:
# Set Seaborn styles
sns.set_style('whitegrid')
sns.set(rc={'figure.figsize':(11.7,8.27)})



In [0]:
#@title Spending by category

# form fields
categories = ["eating_out","general"]  #@param
max_income = 400  #@param {type: "slider", min: 0, max: 5000}
max_spend = -500 #@param {type: "slider", min: -2000, max: 0}

# filter data
tmp = df.loc[(df['local_amount']>max_spend) & (df['local_amount']<max_income) & (df['category_comb'].isin(categories))]
tmp.head()

  
# plot
sns.catplot(x = "category_comb", y = "local_amount", kind = "swarm", data = tmp)
fig = plt.gcf()
fig.set_size_inches( 16, 10)
plt.xticks(rotation=45)

In [0]:
#@title Spending by category

# form fields
categories = ["eating_out","general","transport","holidays"]  #@param
max_income = 400  #@param {type: "slider", min: 0, max: 5000}
max_spend = -500 #@param {type: "slider", min: -2000, max: 0}

# filter data
tmp = df.loc[(df['local_amount']>max_spend) & (df['local_amount']<max_income) & (df['category_comb'].isin(categories))]
tmp.head()

  
# plot
sns.catplot(x = "category_comb", y = "local_amount", kind = "strip", data = tmp, alpha = 0.7)
fig = plt.gcf()
fig.set_size_inches( 16, 10)
plt.xticks(rotation=45)

In [0]:
# Shows how to create a double list 

categories = df['category'].unique()
avg_values = df.groupby('category').agg({'amount':'mean'}).amount
[{'label': i, 'value': j} for i,j in zip(categories,avg_values)]
dff = pd.DataFrame([{'label': i, 'value': j} for i,j in zip(categories,avg_values)])

In [0]:
dff.head()

In [0]:
categories = df['category'].unique()
avg_values = df.groupby('category').agg({'amount':'mean'}).amount
test = pd.DataFrame(data = {'cat':categories,'value':avg_values})

In [0]:
categories

In [0]:
avg_values

In [0]:
np.isnan(np.nan)

True

# Categorize outstanding transactions

In [0]:
df.category.unique()

array(['bills', 'cash', 'eating_out', 'entertainment', 'general',
       'expenses', 'groceries', 'holidays', 'personal_care', 'shopping',
       'transport', nan], dtype=object)

In [0]:
df.loc[(pd.isnull(df.category)) & (df.amount > 1500) & (df.category != "transfer"), 'category'] = 'salary'

In [0]:
df[pd.isnull(df.category)].sort_values('amount',ascending = 1)


,id,date,created,amount,currency,local_amount,local_currency,category,emoji,description,address,notes,alt_category,category_comb,exclude,ymd


In [0]:
df[df.category == 'expenses'].sort_values('amount',ascending = 1)

,id,date,created,amount,currency,local_amount,local_currency,category,emoji,description,address,notes,alt_category,category_comb,exclude,ymd


In [0]:
# Impliment simple changes 

def category_assignment(df): 
  '''This function assigns categories where the category value
  is not available. It follows fairly simple rules. Currently 
  implimented ones are: 
  
  1.Transfer to and Transfer from are assigned 'transfer'
  2.a 16 string number is 'general'
  3.Funding C is 'transfer'
  4.>2000 income is salary (if not also transfer)
  5.Faster payments is asigned to entertainment
  6.If notes contains funding cirlce then modify to Transfer
  7.
  '''
 
  # Bulk classification
  df.loc[df.description.str.contains("Transfer to|Transfer from"), 'category'] = 'transfer' # transfers
  df.loc[(df.description.str.contains("Maria Lachenauer") & (df.amount < 0)), 'category'] = 'transfer' # Isabel transfers
  df.loc[(df.description.str.contains("Robert Sturrock") & (df.amount < 0)), 'category'] = 'transfer' # Isabel transfers
  df.loc[(df.description.str.contains("Robert Sturrock") & (df.amount < 0)), 'category'] = 'transfer' # Isabel transfers  
  df.loc[df.description.str.match("\d{16}"), 'category'] = 'general' # credit card
  df.loc[df.description.str.contains("Funding C"), 'category'] = 'transfer' # funding circle
  df.loc[df.description.str.match("VG\d{16}"), 'category'] = 'transfer' # vangaurd
  df.loc[(df.description.str.contains("Faster Payments")) & (df.amount < abs(500)), 'category'] = 'general' # payments to others

  # Tag specifics
  df.loc[df.description.str.contains("#savings"), 'category'] = 'transfer' # payments to others
  df.loc[df.description.str.contains("#savings"), 'category'] = 'transfer' # payments to others
  df.loc[df.description.str.contains("#google"), 'category'] = 'transfer' # payments to others

  
  # This is replacing just the nulls
  df.loc[(pd.isnull(df.category)) & (df.amount > 1500), 'category'] = 'salary'
  df.loc[(pd.isnull(df.category)), 'category'] = 'general'
  return df

In [0]:
df = category_assignment(df)



In [0]:
df.category.unique()

array(['bills', 'general', 'cash', 'transfer', 'eating_out',
       'entertainment', 'expenses', 'groceries', 'holidays',
       'personal_care', 'shopping', 'transport', 'salary', nan],
      dtype=object)